In [110]:
import numpy as np
import pandas as pd
from mido import MidiFile
from collections import namedtuple

In [112]:
midi_dir = "../midi/"
TimedNote = namedtuple('TimedNote', ['note', 'start_tick', 'end_tick'])
CustomMessage = namedtuple('CustomMessage', ['start_tick', 'note', 'on'])

In [82]:
"""
Structure for keeping track of all notes playing at the same time
"""
class ActiveNotes:
    def __init__(self):
        self.cur_tick = 0
        self.currently_playing = np.zeros([128], dtype='bool')
        # Array to prevent adding the same highest note twice
        # (f.e. some note gets turned off and some note which has been added to the melody already
        # might still be playing and gets evaluated as highest note again)
        self.already_in_melody = np.zeros([128], dtype='bool')

    def note_on(self, note):
        self.currently_playing[note] = True

    def note_off(self, note):
        self.currently_playing[note] = False
        self.already_in_melody[note] = False

    """
    Return highest note and ticks when it started/stopped
    delta_tick: time since latest MIDI Message
    """
    def get_highest_note(self, delta_tick):
        previous_tick = self.cur_tick
        self.cur_tick = self.cur_tick + delta_tick
        for i in range(127, -1, -1):
            if self.currently_playing[i] and not self.already_in_melody[i]:
                self.already_in_melody[i] = True
                return TimedNote(i, previous_tick, self.cur_tick)

        return TimedNote(-1, previous_tick, self.cur_tick)


In [121]:
def get_highest_melody(file, tracks_to_merge, tick_ignore):
    mid = MidiFile(midi_dir + file)
    highest_melodies = []
    merged_melody = []

    # Process all tracks separately first
    for track_no in tracks_to_merge:
        an = ActiveNotes()
        highest_melody = []
        additive_tick = 0
        for msg in mid.tracks[track_no]:
            if msg.type == 'control_change':
                # What the fuck, there can be TWO control change messages in a row lol
                additive_tick = additive_tick + msg.time

            if msg.type == 'note_on':
                # Not instant; some time has passed => evaluate previous highest note
                tick = msg.time + additive_tick
                additive_tick = 0
                if tick != 0:
                    res = an.get_highest_note(tick)
                    # Filter out cases when nothing is playing
                    if res.note != -1:
                        # highest_melody.append(res)
                        # Add turn on/turn off messages
                        highest_melody.append(CustomMessage(res.start_tick, res.note, 1))
                        highest_melody.append(CustomMessage(res.end_tick, res.note, 0))

                if msg.velocity != 0:
                    an.note_on(msg.note)
                else:
                    an.note_off(msg.note)

        highest_melodies.append(highest_melody)

    # Merge all highest melody messages into one list
    mega_list = []
    for i in highest_melodies:
        mega_list = mega_list + i

    # Sort all messages by starting tick
    mega_list.sort(key=lambda custom_msg: custom_msg.start_tick)
    an = ActiveNotes()
    cur_tick = mega_list[0][0]

    for custom_msg in mega_list:
        # Treat a very small time difference between messages as if the events happened simultaneously
        # So that the higher note gets judged correctly
        if abs(custom_msg.start_tick - cur_tick) > tick_ignore:
            res = an.get_highest_note(0)
            if res.note != -1:
                merged_melody.append(res.note)

        if custom_msg.on:
            an.note_on(custom_msg.note)

        else:
            an.note_off(custom_msg.note)

        cur_tick = custom_msg.start_tick


    return merged_melody, mega_list


In [122]:
res = get_highest_melody("bach_846.mid", [1, 2], 10)
print(res[0])

[60, 64, 67, 72, 76, 67, 72, 76, 60, 64, 67, 72, 76, 67, 72, 76, 60, 62, 69, 74, 77, 69, 74, 77, 60, 62, 69, 74, 77, 69, 74, 77, 59, 62, 67, 74, 77, 67, 74, 77, 59, 62, 67, 74, 77, 67, 74, 77, 60, 64, 67, 72, 76, 67, 72, 76, 60, 64, 67, 72, 76, 67, 72, 76, 60, 64, 69, 76, 81, 69, 76, 81, 60, 64, 69, 76, 81, 69, 76, 81, 60, 62, 66, 69, 74, 66, 69, 74, 60, 62, 66, 69, 74, 66, 69, 74, 59, 62, 67, 74, 79, 67, 74, 79, 59, 62, 67, 74, 79, 67, 74, 79, 59, 60, 64, 67, 72, 64, 67, 72, 59, 60, 64, 67, 72, 64, 67, 72, 57, 60, 64, 67, 72, 64, 67, 72, 57, 60, 64, 67, 72, 64, 67, 72, 50, 57, 62, 66, 72, 62, 66, 72, 50, 57, 62, 66, 72, 62, 66, 72, 55, 59, 62, 67, 71, 62, 67, 71, 55, 59, 62, 67, 71, 62, 67, 71, 55, 58, 64, 67, 73, 64, 67, 73, 55, 58, 64, 67, 73, 64, 67, 73, 53, 57, 62, 69, 74, 62, 69, 74, 53, 57, 62, 69, 74, 62, 69, 74, 53, 56, 62, 65, 71, 62, 65, 71, 53, 56, 62, 65, 71, 62, 65, 71, 52, 55, 60, 67, 72, 60, 67, 72, 52, 55, 60, 67, 72, 60, 67, 72, 52, 53, 57, 60, 65, 57, 60, 65, 52, 53,

In [98]:
# Load a midi file
midd = MidiFile(midi_dir + "Fr_Elise.mid")
# Print notes in midi track
for track in midd.tracks:
    print(track.name)

print(midd.type)

Piano 
Piano 
1


In [103]:
for msgg in midd.tracks[0]:
    # Not instant; some time has passed => evaluate previous highest note
    print(msgg)

MetaMessage('track_name', name='Piano\x00', time=0)
MetaMessage('time_signature', numerator=1, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='C', time=0)
MetaMessage('set_tempo', tempo=833333, time=0)
control_change channel=0 control=121 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=0 time=0
control_change channel=0 control=93 value=0 time=0
MetaMessage('midi_port', port=0, time=0)
note_on channel=0 note=76 velocity=33 time=0
note_on channel=0 note=76 velocity=0 time=113
note_on channel=0 note=75 velocity=33 time=7
note_on channel=0 note=75 velocity=0 time=113
MetaMessage('time_signature', numerator=3, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=7)
note_on channel=0 note=76 velocity=33 time=0
note_on channel=0 note=76 velocity=0 time=113
note_